In [1]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
books_dir = os.path.join(current_dir, "books")
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

print(f"書籍目錄: {books_dir}")
print(f"持久化目錄: {persistent_directory}")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保書籍目錄存在
    if not os.path.exists(books_dir):
        raise FileNotFoundError(
            f"目錄 {books_dir} 不存在。請檢查路徑。"
        )

    # 列出目錄中所有文字檔案
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    # 從每個檔案讀取文字內容並儲存元數據
    documents = []
    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path)
        book_docs = loader.load()
        for doc in book_docs:
            # 為每個文件添加元數據以指示其來源
            doc.metadata = {"source": book_file}
            documents.append(doc)

    # 將文件分割成塊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立並持久化向量存儲
    print("\n--- 正在建立並持久化向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- 完成建立並持久化向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

書籍目錄: /Users/roberthsu2003/Documents/GitHub/__2025_08_30__chihlee_langchain__/lesson7/books
持久化目錄: /Users/roberthsu2003/Documents/GitHub/__2025_08_30__chihlee_langchain__/lesson7/db/chroma_db_with_metadata_chinese_nb
持久化目錄不存在。正在初始化向量存儲...

--- 文件塊資訊 ---
文件塊數量: 81

--- 正在建立嵌入 ---


/var/folders/7g/kb_wcfhj3710_xnyqdgx_v940000gn/T/ipykernel_13754/2351684797.py:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(



--- 完成建立嵌入 ---

--- 正在建立並持久化向量存儲 ---

--- 完成建立並持久化向量存儲 ---


In [7]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "什麼是紅利點數卡"

# 根據查詢檢索相關文件
# search_type="similarity_score_threshold": 使用相似度分數閾值過濾
# k=3: 返回最多 3 個相關文件
# score_threshold=0.1: 較低的閾值，可以返回更多相關文件
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.3},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    print(f"來源: {doc.metadata['source']}\n")

# # 進階查詢：只從特定來源檢索
# print("\n--- 只從《路由器設定手冊》檢索 ---")
# # 使用 filter 參數指定元數據條件
# specific_docs = db.similarity_search(
#     query,
#     k=3,
#     filter={"source": "路由器設定手冊.txt"}
# )

# for i, doc in enumerate(specific_docs, 1):
#     print(f"文件 {i}:\n{doc.page_content}\n")
#     print(f"來源: {doc.metadata['source']}\n")



--- 相關文件 ---
文件 1:
密碼安全建議：
- 勿使用生日、電話等
- 定期更換密碼
- 不同服務使用不同密碼
- 妥善保管密碼

第二章：紅利點數累積與兌換

2.1 點數累積規則
----------------------------------------
基本累積（紅利點數卡）：
- 一般消費：每 20元 = 1點
- 指定通路：每 10元 = 1點
- 海外消費：每 15元 = 1點

指定通路加碼（每月更新）：
- 便利商店：10元 = 1點
- 量販超市：10元 = 1點
- 加油站：10元 = 1點
- 百貨公司：10元 = 1點
- 線上購物：10元 = 1點

特殊加碼活動：
- 生日當月：點數 2倍送
- 新卡首刷：贈送 3,000點
- 推薦辦卡：每成功推薦贈 1,000點
- 滿額加碼：單筆滿萬元加贈 500點

點數計算範例：
消費 1,000元於一般商店：
1,000 ÷ 20 = 50點

消費 1,000元於超市（指定通路）：
1,000 ÷ 10 = 100點

消費 1,000元於生日當月超市：
(1,000 ÷ 10) × 2 = 200點

不累積點數項目：
- 預借現金
- 繳納稅款、規費
- 繳納電信費、保費（部分銀行）
- 分期付款手續費
- 循環利息
- 年費

2.2 點數查詢
----------------------------------------
查詢管道：

電話查詢：
1. 撥打客服專線 02-8888-9999
2. 選擇「點數查詢」
3. 輸入身分證字號與電話密碼
4. 語音播報點數餘額

網路查詢：
1. 登入網路銀行
2. 點選「信用卡」→「紅利點數」
3. 查看累積點數、即將到期點數

App查詢：
1. 開啟 TW Bank App
2. 首頁顯示點數摘要
3. 點入可查看詳細累兌記錄

電子帳單：
- 每月帳單載明當期累積點數
- 顯示點數餘額
- 提醒即將到期點數

簡訊通知：
- 可設定點數異動通知
- 到期前 30天提醒
- 累積達門檻提醒

來源: 信用卡權益說明.txt

文件 2:
電子帳單：
- 每月帳單載明當期累積點數
- 顯示點數餘額
- 提醒即將到期點數

簡訊通知：
- 可設定點數異動通知
- 到期前 30天提醒
- 累積達門檻提醒

2.3 點數兌

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何保養洗衣機和路由器？"

print(f"❓ 使用者問題: {question}\n")

# 步驟 1: 從多個來源檢索相關文件
print("=" * 70)
print("步驟 1: 從多個來源檢索相關文件")
print("=" * 70)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 取4個相關文件（可能來自不同手冊）
)

retrieved_docs = retriever.invoke(question)

print(f"✅ 找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    source = doc.metadata.get('source', 'Unknown')
    print(f"📄 文件 {i} - 來源: {source}")
    print(f"   內容: {doc.page_content[:80]}...\n")

# 步驟 2: 建立帶有來源標註的 RAG Chain
print("=" * 70)
print("步驟 2: 建立多來源 RAG Chain")
print("=" * 70)

# 定義提示模板（強調來源資訊）
template = """你是一個專業的產品客服助手。請根據以下多個產品手冊的參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

回答要求：
1. 針對不同產品分別說明（如果問題涉及多個產品）
2. 在回答中標註資訊來源（例如：「根據洗衣機使用說明...」）
3. 只根據參考資料回答，不要編造內容
4. 如果某個產品的資訊不足，請說明
5. 用條列式整理，讓使用者容易閱讀

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM
llm = ChatOllama(model="gpt-oss:20b", temperature=0)

# 定義文件格式化函數（包含來源資訊）
def format_docs_with_source(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"【來源 {i}: {source}】\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# 建立完整的 RAG Chain（整合 3_chains 的技巧）
rag_chain = (
    {
        "context": retriever | format_docs_with_source, 
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ 多來源 RAG Chain 建立完成\n")

# 步驟 3: 執行 Chain 並取得整合答案
print("=" * 70)
print("步驟 3: 執行 Chain 取得整合答案")
print("=" * 70)

answer = rag_chain.invoke(question)

print(f"\n🤖 【AI 整合回答】\n")
print(answer)
print()

# 步驟 4: 示範使用元數據過濾的 Chain
print("=" * 70)
print("步驟 4: 只查詢特定來源的 Chain")
print("=" * 70)

specific_question = "路由器如何設定？"
print(f"❓ 問題: {specific_question}\n")

# 創建只查詢路由器手冊的檢索器
def retrieve_from_router_manual(query):
    docs = db.similarity_search(
        query,
        k=3,
        filter={"source": "路由器設定手冊.txt"}
    )
    return docs

# 建立特定來源的 Chain
specific_chain = (
    {
        "context": RunnablePassthrough() | retrieve_from_router_manual | format_docs_with_source,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

specific_answer = specific_chain.invoke(specific_question)
print(f"🤖 【只查路由器手冊的回答】\n")
print(specific_answer)
print()

print("=" * 70)
print("📚 多來源 RAG Chain 的優勢")
print("=" * 70)
print("✅ 可以同時查詢多個產品手冊")
print("✅ 自動標註資訊來源，方便追蹤")
print("✅ 可以用 metadata 過濾特定來源")
print("✅ 整合多個文件的資訊給出完整答案")
print("\n💡 這展示了 Chain 3 中學到的組合技巧在 RAG 中的應用！")

❓ 使用者問題: 如何保養洗衣機和路由器？

步驟 1: 從多個來源檢索相關文件
✅ 找到 4 個相關文件

📄 文件 1 - 來源: 洗衣機使用說明.txt
   內容: 5.2 每週保養
清潔洗劑盒:
1. 將洗劑盒完全拉出(按住中央釋放鈕)
2. 用溫水沖洗各個槽
3. 用舊牙刷清潔細部
4. 擦乾後裝回

清潔門圈橡膠:
1...

📄 文件 2 - 來源: 洗衣機使用說明.txt
   內容: 7.3 維護安全
- 清潔前務必拔掉電源插頭
- 請勿用水直接沖洗洗衣機
- 請勿使用強酸強鹼清潔劑
- 維修時請聯絡專業人員

7.4 緊急處理
發生以下情況...

📄 文件 3 - 來源: 路由器設定手冊.txt
   內容: 步驟 5：重置路由器
最後手段（將恢復原廠設定）：
1. 找到路由器背面 Reset孔
2. 使用迴紋針按住 10秒
3. 指示燈全亮後放開
4. 等待重新啟動...

📄 文件 4 - 來源: 洗衣機使用說明.txt
   內容: 問題:漏水
可能原因與解決方法:
1. 檢查進水管接頭是否鎖緊
2. 檢查排水管是否正確安裝
3. 檢查洗劑盒是否正確關閉
4. 檢查使用的洗劑是否過量(泡沫溢...

步驟 2: 建立多來源 RAG Chain
✅ 多來源 RAG Chain 建立完成

步驟 3: 執行 Chain 取得整合答案

🤖 【AI 整合回答】

**洗衣機保養（根據《洗衣機使用說明》）**

| 頻率 | 主要項目 | 具體步驟 | 參考來源 |
|------|----------|----------|----------|
| **每週** | 洗劑盒清潔 | 1. 拉出洗劑盒（按住中央釋放鈕）<br>2. 用溫水沖洗各槽<br>3. 用舊牙刷清潔細部<br>4. 擦乾後裝回 | 【來源 1】 |
| | 門圈橡膠清潔 | 1. 拉開橡膠圈<br>2. 用濕布擦拭內側摺疊處<br>3. 檢查硬幣、髮夾等異物<br>4. 用乾布擦乾 | 【來源 1】 |
| **每月** | 排水濾網清潔 | 1. 關閉電源並拔掉插頭<br>2. 打開右下角維修蓋板<br>3. 準備淺盤接水<br>4. 逆時針旋轉濾網蓋取出<br>5. 清除棉絮、異物<br>6. 用清水沖洗<br>7. 檢查底座、清除積水<br>8. 裝回並鎖緊 | 